In [1]:
import os
import pandas as pd
import numpy as np
import json

In [6]:
json_files = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg'
jsons = [os.path.join(json_files, x) for x in os.listdir(json_files) if x.endswith('json')]

In [7]:
jsons

['\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_14_1F_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_23_4O_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_29_3M_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_30_3D_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_34_3L_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\cropped_nucl_seg\\PD_36_5O_cropped.json',
 '\\\\10.162.80.16\\Andre_expansion\\data\\PDAC purity\\NDPI\\pdac purity\\StarDist_2_23_24\\ndpi images\\croppe

In [8]:
# functions to adjust format of contours list

def fix_contours(contours):
    contours_fixed = []
    for polygon in contours:
        print(polygon)
        coords = np.array([list(zip(x,y)) for x,y in [polygon[0]]], dtype=np.int32)
        contours_fixed.append(coords)
    contours_fixed = np.array(contours_fixed)
    return contours_fixed


def adjust_contours(contour, crop_x, crop_y):

    for i, xy in enumerate(contour):
        x = xy[0] - crop_x
        y = xy[1] - crop_y
        
        
        contour[i] = [x, y]
    return contour

In [9]:
import cv2

def cntarea(cnt):
    cnt = np.array(cnt).astype(np.float32)
    area = cv2.contourArea(cnt)
    return area

In [13]:
contour

array([[424.12381114, 423.16800059, 422.11184825, 421.65799083,
        421.24063243, 421.67556895, 421.25576915, 421.21267833,
        421.2062086 , 421.54495372, 421.19986095, 421.87283458,
        422.42288341, 423.54177989, 424.27700938, 425.12771739,
        426.00137462, 426.75808849, 427.92581309, 428.52456798,
        429.60708751, 429.64969005, 430.3143629 , 430.37771739,
        430.10891856, 429.43618907, 429.35354747, 428.42740001,
        427.89285411, 426.87527599, 426.09317149, 425.12771739],
       [ 75.25792572,  74.9422519 ,  74.72447846,  73.68077729,
         72.80821869,  71.64098237,  70.9813144 ,  70.21105072,
         69.43102143,  68.72691987,  67.58653901,  66.95616791,
         66.16295502,  66.38243744,  65.9344394 ,  65.60851166,
         65.8184726 ,  66.27501557,  66.02330658,  66.81432221,
         67.2181308 ,  68.33800385,  69.17931244,  70.21105072,
         71.20177338,  71.99571869,  73.03478119,  73.51085541,
         74.34947846,  74.43004487,  75

In [14]:
outpth= r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\pkl'

for i, json_f_name in enumerate(jsons):
    
    nm = json_f_name.split('\\')[-1].split('.')[0]
    
    outnm = os.path.join(outpth, f'{nm}.pkl')
    
    if not os.path.exists(outnm):
        
        print(json_f_name)
        
        try:
            segmentation_data = json.load(open(json_f_name))
        except:
            print(f'error reading json... Skipping {json_f_name}')
            continue
    
        # centroids = [nuc['centroid'][0] for nuc in segmentation_data]
        centroids = [nuc['centroid'] for nuc in segmentation_data]
        contours = [nuc['contour'] for nuc in segmentation_data]
        contours_fixed = np.array(contours)#fix_contours(contours)
        centroids_np = np.array(centroids)  # for other formatting
        contours_np = np.array(contours)
    
        np_centroids = np.array(centroids)
        
        areas = []
        
        for j in range(len(contours_fixed)):
            
            centroid = centroids[j]
            contour = contours_np[j]#.transpose()  # used for other stuff, too lazy to make formatting the same
            
            cent_x = np_centroids[j,0]
            cent_y = np_centroids[j,1]
            
            area = cntarea(contour)
            areas.append(area)
            
        dat = {
            'Centroid_x': np_centroids[:,1],
            'Centroid_y': np_centroids[:,0],
            'Area': areas,
        }
    
        df = pd.DataFrame(dat).astype(np.float32)  # save a little space with float16 type -> Edit 2 months later, this did not save time.
        
        df.to_pickle(outnm)
        #break
    else:
        print('skipping')

\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_14_1F_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_23_4O_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_29_3M_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_30_3D_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_34_3L_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_36_5O_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\PD_38_9R_cropped.json
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\Star

In [15]:
df.head()

,Centroid_x,Centroid_y,Area
0,5433.200195,86.961052,92.210693
1,5515.200195,72.961052,85.487488
2,5563.200195,78.961052,93.862411
3,5585.200195,76.961052,96.084427
4,5651.200195,108.961052,97.945976


In [16]:
src = outpth
dfs = sorted([_ for _ in os.listdir(src) if _.endswith('.pkl')])

In [17]:
print(len(dfs))
print(dfs)

10
['PD_14_1F_cropped.pkl', 'PD_23_4O_cropped.pkl', 'PD_29_3M_cropped.pkl', 'PD_30_3D_cropped.pkl', 'PD_34_3L_cropped.pkl', 'PD_36_5O_cropped.pkl', 'PD_38_9R_cropped.pkl', 'PD_44_7HH_cropped.pkl', 'PD_45_4M_cropped.pkl', 'PD_6_3P_AMP_cropped.pkl']


In [18]:
import pickle
from scipy.io import savemat

In [19]:
for dfnm in dfs:
    print("Saving: {}".format(dfnm))
    dst = os.path.join(*[os.path.dirname(src),'mat',dfnm.replace('pkl','mat')])
    if os.path.exists(dst):
        print("mat already exists, skipping the file ID {}".format(dfnm))
        continue
        
    with open(os.path.join(src, dfnm), 'rb') as f:
        df = pickle.load(f)

    # df = pd.read_pickle(os.path.join(src,dfnm))
    # df = df[['Centroid_x', 'Centroid_y']]
    col_names = df.columns.tolist()
    df = [_ for _ in df.to_numpy()]
    df = np.array(df)
    print(dst)
    
    savemat(dst, {'xy':df})

Saving: PD_14_1F_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_14_1F_cropped.mat
Saving: PD_23_4O_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_23_4O_cropped.mat
Saving: PD_29_3M_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_29_3M_cropped.mat
Saving: PD_30_3D_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_30_3D_cropped.mat
Saving: PD_34_3L_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_34_3L_cropped.mat
Saving: PD_36_5O_cropped.pkl
\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity\StarDist_2_23_24\ndpi images\cropped_nucl_seg\mat\PD_36_5O_cropped.mat
Saving: PD